# Step 1: Loading models

In [1]:
from keras import models

In [ ]:
# Load RGB and flow model previously trained
rgb_model = models.load_model('checkpoints/weights_i3drgb.hdf5')
flow_model = models.load_model('checkpoints/weights_i3dflow.hdf5')

In [63]:
# Get rid of last layer in each model (softmax layer)
rgb_model.pop()
flow_model.pop()

In [64]:
# Summary of rgb model
rgb_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
i3d_inception (Functional)   (None, None, 1, 1, 1024)  12294544  
_________________________________________________________________
dropout (Dropout)            (None, None, 1, 1, 1024)  0         
_________________________________________________________________
Conv3d_6a_1x1 (Conv3D)       (None, None, 1, 1, 2)     2048      
_________________________________________________________________
reshape (Reshape)            (None, None, 2)           0         
_________________________________________________________________
lambda (Lambda)              (None, 2)                 0         
Total params: 12,296,592
Trainable params: 2,785,520
Non-trainable params: 9,511,072
_________________________________________________________________


In [65]:
# Summary of flow model
flow_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
i3d_inception (Functional)   (None, None, 1, 1, 1024)  12272592  
_________________________________________________________________
dropout (Dropout)            (None, None, 1, 1, 1024)  0         
_________________________________________________________________
Conv3d_6a_1x1 (Conv3D)       (None, None, 1, 1, 2)     2048      
_________________________________________________________________
reshape (Reshape)            (None, None, 2)           0         
_________________________________________________________________
lambda (Lambda)              (None, 2)                 0         
Total params: 12,274,640
Trainable params: 2,785,520
Non-trainable params: 9,489,120
_________________________________________________________________


# Step 2: Create data generators

In [68]:
# Parameters
batch_size = 8
rgb_path_test = '../datai3d/rgb/test/'
flow_path_test = '../datai3d/flow/test/'

In [ ]:
# Create data generators for test set (rgb and flow)
rgb_test_generator = DataGenerator(directory=rgb_path_test,
                               shuffle = False,
                                batch_size=batch_size, 
                                data_augmentation=False,
                                target_frames = None,
                                flip = False)

flow_test_generator = DataGenerator(directory=flow_path_test,
                               shuffle = False,
                                batch_size=batch_size, 
                                data_augmentation=False,
                                target_frames = None,
                                flip = False)

# Step 3: Get predictions

In [70]:
# Get the output of rgb model
rgb_predictions = rgb_model.predict(x = rgb_test_generator,
                                    steps = rgb_test_generator.n_files//batch_size,
                                   verbose = 1)

50/50 [==============================] - 384s 8s/step


In [71]:
# Get the output of flow model
flow_predictions = flow_model.predict(x = flow_test_generator,
                                    steps = flow_test_generator.n_files//batch_size,
                                   verbose = 1)

50/50 [==============================] - 328s 7s/step


In [72]:
# Combine both outputs to get the predictions
predictions = (rgb_predictions + flow_predictions).argmax(-1)

# Step 4: Evaluate model

In [78]:
from sklearn import metrics

In [76]:
# Get the ground truth from the data generator
ground_truth = list(rgb_test_generator.Y_dict.values())
ground_truth = np.array([labels.argmax(-1) for labels in ground_truth])

In [81]:
# Print accuracy
metrics.accuracy_score(ground_truth, predictions)

0.785

In [80]:
# Print classification report
print(metrics.classification_report(ground_truth, predictions))

              precision    recall  f1-score   support

           0       0.76      0.83      0.80       200
           1       0.82      0.73      0.77       200

    accuracy                           0.79       400
   macro avg       0.79      0.78      0.78       400
weighted avg       0.79      0.79      0.78       400

